<font style="font-size:96px; font-weight:bolder; color:#0040a0"><img src="http://montage.ipac.caltech.edu/docs/M51_logo.png" alt="M" style="float: left; padding: 25px 30px 25px 0px;" /></font>

<i><b>Montage</b> is a general astronomical image toolkit with facilities for reprojection, background matching, coaddition and visualization.  It can be used as a set of command-line tools (Linux, OS X and Windows), C library calls (Linux and OS X) and as Python binary extension modules.</i>
    
<i>Montage source code can be downloaded from GitHub ( https://github.com/Caltech-IPAC/Montage ).  The Python package can be installed from PyPI ("pip install MontagePy"). See
http://montage.ipac.caltech.edu/ for more information.</i>

# MontagePy.main modules: mSubCube

Most of Montage is focused on 2D astronomical image mosaics: reprojection, background matching, coaddition and so on.  But there is also a need for tools that operate on data "cubes": three- or four-dimensional arrays where two of the axes represent the same projected sky coordinates as we find in the 2D images.  Usually, the third axis is some form of wavelength.

The problem sets are not completely parallel: datacubes do not generally need background matching but you do frequently want cutouts in the none spatial dimensions and to transpose axes.

Montage includes a set of routines for manipulating datacubes:

<ul>
<li><b>mmProjectCube</b> &mdash; Reproject the spatial dimensions.</li>
<li><b>mAddCube</b> &mdash; Reproject the cube.</li>
<li><b>mShrinkCube</b> &mdash; Rescale a cube (integer scaling in the non-spatial dimensions).</li>
<li><b>mSubCube</b> &mdash;Cut a portion out of a cube.</li>
<li><b>mTranspose</b> &mdash; Transpose a cube's axes.</li>
</ul>

This routine, mSubCube, cuts a multi-dimensional box out of a cube.  In the spatial dimensions this is exactly the same as mSubimage: a sky location and size or pixel ranges.  In the third (and fourth) dimensions it is just index ranges (or an index value list).
 
 <b>Note:</b> The MontagePy python package has no external dependencies. We include other utilities on this page to aid in visualizing MontagePy package results.


In [1]:
from MontagePy.main import mSubCube, mViewer

help(mSubCube)

Help on built-in function mSubCube in module MontagePy.main:

mSubCube(...)
    mSubCube extracts a sub-cube from one FITS file into a new FITS file.
    
    Parameters
    ----------
    mode : int
        Processing mode. The two main modes are 0 (SKY) and 1 (PIX), corresponding to cutouts are in sky coordinate or pixel space. The two other modes are 3 (HDU) and 4 (SHRINK), where the region parameters are ignored and you get back either a single HDU or an image that has had all the blank border pixels removed.
    infile : str
        Input FITS file.
    outfile : str
        Sub-cube output FITS file.
    ra : float
        RA of cutout center (or start X pixel in PIX mode.
    dec : float
        Dec of cutout center (or start X pixel in PIX mode.
    xsize : float
        X size in degrees (SKY mode) or pixels (PIX mode).
    ysize : float
        Y size in degrees (SKY mode) or pixels (PIX mode).
    hdu : int, optional
        Optional HDU offset for input file.
    nowcs : bo

## mSubCube Example

In this example, we will cut a region, in both space and wavelength from one of the inputs to our GALFA mosaic.

The data used in the datacube Jupyter pages come from the Galactic Arecibo L-band Feed Array HI (GALFA-HI) survey (Peek et al., 2011, Ap J Suppl, 194, 20; DOI 10.1088/0067-0049/194/2/20; ADS Bibcode 2011ApJS..194...20P).

In [12]:
rtn = mSubCube(0, "GALFA/shrunken/GALFA_HI_RA+DEC_012.00+10.35_N.fits",
              "work/GALFA/GALFAsubcube.fits",
              12., 10., 5., 5.)
print(rtn)

{'status': '0', 'content': b'normal', 'warning': b'Check CDELT, CRPIX values for axes 3 and 4.'}


## Before and After

Here are the original image and the cutout. Since these are cubes, we have to collapse it in the third dimension for display. 


In [19]:
from IPython.display import HTML, display, Image

rtn = mViewer('-color yellow -grid eq j2000 \
               -ct 4 -gray "GALFA/shrunken/GALFA_HI_RA+DEC_012.00+10.35_N.fits[0][60,68]" \
               -2s max gaussian-log -out work/GALFA/GALFA_HI_RA+DEC_012.00+10.35_N_subcube.png', 
              '', mode=2 )

rtn = mViewer('-color yellow -grid eq j2000 \
               -ct 4 -gray "work/GALFA/GALFAsubcube.fits[0][60,68]" \
               -2s max gaussian-log -out work/GALFA/GALFAsubcube.png', 
              '', mode=2 )

display(HTML("<table><tr><td><img src='work/GALFA/GALFA_HI_RA+DEC_012.00+10.35_N_subcube.png'></td> \
                         <td><img src='work/GALFA/GALFAsubcube.png'></td></tr></table>"))

,


&nbsp;<p/>

## mSubCube Error Handling

If mSubCube encounters an error, the return structure will just have two elements: a status of 1 ("error") and a message string that tries to diagnose the reason for the error.

For instance, if the user specifies a datacube that doesn't exist: 

In [20]:
rtn = mSubCube(0, "GALFA/shrunken/unknown.fits",
              "work/GALFA/GALFAsubcube.fits",
              12., 10., 5., 5.)
print(rtn)

{'status': '1', 'msg': b'File GALFA/shrunken/unknown.fits not found.'}


&nbsp;



# Classic Montage: mSubCube as a Stand-Alone Program


### mSubCube Unix/Windows Command-line Arguments

<p>mSubCube can also be run as a command-line tool in Linux, OS X, and Windows:</p>

<p><tt>
<b>Usage:</b> mSubCube [-D3 selection-list][-D4 selection-list][-d][-a(ll pixels)][-h hdu][-s statusfile] in.fit out.fit ra dec xsize [ysize] | mSubCube -p [-D3 selection-list][-D4 selection-list][-d][-h hdu][-s statusfile] in.fit out.fit xstartpix ystartpix xpixsize [ypixsize] | mSubCube -c [-D3 selection-list][-D4 selection-list][-d][-h hdu][-s statusfile] in.fit out.fit
</tt></p>
<p>&nbsp;</p>
<p>If you are writing in C/C++, mSubCube can be accessed as a library function:</p>


<pre>
/*-***********************************************************************/
/*                                                                       */
/*  mSubimage                                                            */
/*                                                                       */
/*  This program subsets an input image around a location of interest    */
/*  and creates a new output image consisting of just those pixels.      */
/*  The location is defined by the RA,Dec (J2000) of the new center and  */
/*  the XY size in degrees of the area (X and Y) in the direction of     */
/*  the image axes, not Equatorial coordinates.                          */
/*                                                                       */
/*   int    mode           Processing mode. The two main modes are       */
/*                         0 (SKY) and 1 (PIX), corresponding to cutouts */
/*                         are in sky coordinate or pixel space. The two */
/*                         other modes are 3 (HDU) and 4 (SHRINK), where */
/*                         the region parameters are ignored and you get */
/*                         back either a single HDU or an image that has */
/*                         had all the blank border pixels removed.      */
/*                                                                       */
/*   char  *infile         Input FITS file                               */
/*   char  *outfile        Subimage output FITS file                     */
/*                                                                       */
/*   double ra             RA of cutout center (or start X pixel in      */
/*                         PIX mode                                      */
/*   double dec            Dec of cutout center (or start Y pixel in     */
/*                         PIX mode                                      */
/*                                                                       */
/*   double xsize          X size in degrees (SKY mode) or pixels        */
/*                         (PIX mode)                                    */
/*   double ysize          Y size in degrees (SKY mode) or pixels        */
/*                         (PIX mode)                                    */
/*                                                                       */
/*   int    hdu            Optional HDU offset for input file            */
/*   int    nowcs          Indicates that the image has no WCS info      */
/*                         (only makes sense in PIX mode)                */
/*                                                                       */
/*   char  *d3constraint   String describing the datacube third          */
/*                         dimension selection constraints               */
/*                                                                       */
/*   char  *d4constraint   String describing the datacube fourth         */
/*                         dimension selection constraints               */
/*                                                                       */
/*   int    debug          Debugging output level                        */
/*                                                                       */
/*************************************************************************/

struct mSubCubeReturn *mSubCube(int mode, char *infile, char *outfile, double ra, double dec,
                                double xsize, double ysize, int hdu, int nowcs, char *d3constraint, 
                                char *d4constraint, int debugin)
</pre>
<p><b>Return Structure</b></p>
<pre>
struct mSubCubeReturn
{
   int    status;        // Return status (0: OK, 1:ERROR)
   char   msg    [1024]; // Return message (for error return)
   char   json   [4096]; // Return parameters as JSON string
   char   content[1024]; // String giving an idea of output content (e.g., 'blank', 'flat', or 'normal'.   
   char   warning[1024]; // If warranted, warning message about CDELT, CRPIX, etc.   
};
</pre>